# Selecting a calibration source for gage ball density

In this example, the density of a gage ball must be calibrated by an external calibration lab. The lab must be selected to meet or exceed density uncertainty requirements. The nominal values are:

Parameter | Value
----------|------
mass $m$     | 86.030 g
diameter $d$  | 22.225 mm
density $\rho$  | 14.967 g/cm$^3$
required $u(\rho)$ | 0.04 g/cm$^3$ (k=2)

Density is determined by the measurement model:

$\rho = \frac{6m}{\pi d^3}$

This is a reverse uncertainty propagation problem, as the uncertainty of the output is known, and we need to determine the uncertainty of the input variables to meet that requirement. However, there are two input variables to work with, so there are two unknowns. To find the combined requirements for measuring mass and diameter, a reverse-sweep calculation can be done. Specify a range of possible $u(d)$ values and solve for the corresponding $u(m)$ that meets the density uncertainty requirement.


In [1]:
from suncal import reverse, sweep

### Set up the reverse calculation

First, set up a reverse propagation calculation using fixed values for $u(m)$ and $u(d)$. Be sure to include the `units` parameter to allow the calculator to perform the proper conversions. Here, mass is entered in mg, diameter in microns, and the output density value is converted to g/cm$^3$. Also, the `targetunc` value is entered as a k=1 standard uncertainty.

You could run `.calculate()` on `rcalc` to see the formulas being solved before starting the sweep. 

In [2]:
rcalc = reverse.ModelReverse('rho = 6*m/(pi*d**3)', solvefor='m', targetnom=14.967, targetunc=.02, targetunits='g/cm^3')
rcalc.var('m').measure(86030, units='mg').typeb(std=10, units='mg')
rcalc.var('d').measure(22225, units='um').typeb(std=5, units='um')
rcalc.calculate()

## GUM reverse uncertainty

$\rho = \frac{6 m}{\pi d^{3}}$

 Combined uncertainty:

$u_{\rho} = \frac{6 \sqrt{\frac{d^{2} u_{m}^{2} + 9 m^{2} u_{d}^{2}}{d^{8}}}}{\pi}$

solved for uncertainty of input:

$u_{m} = \frac{\sqrt{\pi^{2} d^{8} u_{\rho}^{2} - 9 \pi^{2} d^{6} \rho^{2} u_{d}^{2}}}{6 d}$

 For output value of 
14.967 g/cm³
 ± 
0.020 g/cm³
 (k=1),

required input value is 
86032 mg
 ± 
99 mg
 (k=1).



---

## Monte Carlo reverse uncertainty

For output value of 
14.967 g/cm³
 ± 
0.020 g/cm³
 (k=1), required input value is: 
86032 mg
 ± 
99 mg
 (k=1).

### Sweep the diameter uncertainty

Now, use the `rcalc` set up in the previous cell as the input to a `UncertSweepReverse`. Sweep the diameter uncertainty from 1 to 9. The `summary_withplots()` report will show the results of each sweep point in a table and a plot. Any calibration provider that can provide uncertainties below the curve is acceptable to meet the density uncertainty requirement.

In [3]:
s = sweep.UncertSweepReverse('rho = 6*m/(pi*d**3)', solvefor='m', targetnom=14.967, targetunc=.02, targetunits='g/cm^3')
s.model.var('m').measure(86030, units='mg').typeb(std=10, units='mg')
s.model.var('d').measure(22225, units='um').typeb(std=5, units='um')

s.add_sweep_unc('d', list(range(9)), comp='Type B')
result = s.calculate()
result.report.summary_withplots()

$Type B$,$m$,$u_{m}$
0.0,86000 mg,110 mg
1.0,86000 mg,110 mg
2.0,86000 mg,110 mg
3.0,86000 mg,110 mg
4.0,86000 mg,110 mg
5.0,86000 mg,99 mg
6.0,86000 mg,91 mg
7.0,86000 mg,81 mg
8.0,86000 mg,68 mg
$Type B$,$m$,$u_{m}$
